# NRL match analysis

Going to determine which stats have the biggest impact on the outcome of NRL matches. We will see if we can use this information to make predictions about future matches.

In [51]:
#Import Pandas
import pandas as pd

In [88]:
#Connect to database
import mysql.connector as sql

mydb = sql.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)


In [89]:
#Function to easily create dataframes from SQL query results
def create_df(query):
    return pd.read_sql_query(query, con=mydb)

In [91]:
#Only want to analyze matches from the 2018 season
matches2018 = "SELECT * FROM Matches WHERE year(date) = '2018';"
matches2018 = create_df(matches2018)
matches2018.head()

,id,date,round,home_team_id,home_score,away_team_id,away_score,winner,is_draw,stadium_id,weather,url
0,1,2018-03-08,1,14,34,1,12,14,0,1,None,http://www.nrl.com/draw/nrl-premiership/2018/r...
1,2,2018-03-09,1,8,19,6,18,8,0,2,None,http://www.nrl.com/draw/nrl-premiership/2018/r...
2,3,2018-03-09,1,10,20,4,14,10,0,3,None,http://www.nrl.com/draw/nrl-premiership/2018/r...
3,4,2018-03-10,1,16,10,15,8,16,0,4,None,http://www.nrl.com/draw/nrl-premiership/2018/r...
4,5,2018-03-10,1,13,20,9,32,9,0,5,None,http://www.nrl.com/draw/nrl-premiership/2018/r...


In [92]:
matches2018 = matches2018.drop(["stadium_id", "weather", "url"], axis=1)
matches2018.head()

,id,date,round,home_team_id,home_score,away_team_id,away_score,winner,is_draw
0,1,2018-03-08,1,14,34,1,12,14,0
1,2,2018-03-09,1,8,19,6,18,8,0
2,3,2018-03-09,1,10,20,4,14,10,0
3,4,2018-03-10,1,16,10,15,8,16,0
4,5,2018-03-10,1,13,20,9,32,9,0


In [93]:
def find_losing_team(row):
    if row["winner"] == row["home_team_id"]:
        return row["away_team_id"]
    elif row["winner"] == row["away_team_id"]:
        return row["home_team_id"]
    else:
        return None
    
def find_winning_points(row):
    if row["winner"] == row["home_team_id"]:
        return row["home_score"]
    elif row["winner"] == row["away_team_id"]:
        return row["away_score"]
    else:
        return None

def find_losing_points(row):
    if row["loser"] == row["home_team_id"]:
        return row["home_score"]
    elif row["loser"] == row["away_team_id"]:
        return row["away_score"]
    else:
        return None
    

matches2018['loser'] = matches2018.apply(lambda row: find_losing_team(row), axis=1)
matches2018['win_score'] = matches2018.apply(lambda row: find_winning_points(row), axis=1)
matches2018['lose_score'] = matches2018.apply(lambda row: find_losing_points(row), axis=1)
matches2018 = matches2018[['winner', 'loser', 'is_draw', 'win_score', 'lose_score', 'home_team_id', 'away_team_id',
                                         'home_score', 'away_score', 'date', 'round']]
matches2018.head()


,winner,loser,is_draw,win_score,lose_score,home_team_id,away_team_id,home_score,away_score,date,round
0,14,1,0,34,12,14,1,34,12,2018-03-08,1
1,8,6,0,19,18,8,6,19,18,2018-03-09,1
2,10,4,0,20,14,10,4,20,14,2018-03-09,1
3,16,15,0,10,8,16,15,10,8,2018-03-10,1
4,9,13,0,32,20,13,9,20,32,2018-03-10,1


In [110]:
winning_scores = matches2018.groupby(['winner'])[['win_score']].sum().sort_values('win_score', ascending=False)
win_totals = matches2018.groupby('winner').count()

#losing_scores = matches2018.groupby(['loser'])[['lose_score']].sum().sort_values('lose_score', ascending=False)
#total_wins = matches2018['winner'].value_counts()
#total_loses = matches2018['loser'].value_counts()
win_totals

,loser,is_draw,win_score,lose_score,home_team_id,away_team_id,home_score,away_score,date,round
winner,,,,,,,,,,
1,15,15,15,15,15,15,15,15,15,15
2,10,10,10,10,10,10,10,10,10,10
3,8,8,8,8,8,8,8,8,8,8
4,16,16,16,16,16,16,16,16,16,16
5,8,8,8,8,8,8,8,8,8,8
6,7,7,7,7,7,7,7,7,7,7
7,16,16,16,16,16,16,16,16,16,16
8,9,9,9,9,9,9,9,9,9,9
9,15,15,15,15,15,15,15,15,15,15


In [19]:
#Make final standings
standings2018 = '''SELECT t.nickname team, 
	count(m.winner) * 2 as points,
    count(m.winner) as wins
FROM Matches m
JOIN Teams t
ON m.winner = t.id
WHERE year(m.date) = 2018
GROUP BY m.winner
ORDER BY points DESC;'''
standings2018 = create_df(standings2018)
print(standings2018)

          team  points  wins
0     Roosters      32    16
1    Rabbitohs      32    16
2        Storm      32    16
3       Sharks      32    16
4      Dragons      30    15
5      Broncos      30    15
6     Warriors      30    15
7     Panthers      30    15
8       Tigers      24    12
9      Raiders      20    10
10     Knights      18     9
11      Titans      16     8
12     Cowboys      16     8
13    Bulldogs      16     8
14  Sea Eagles      14     7
15        Eels      12     6


In [ ]:
#PlayerMatchStats
stats2018 = '''SELECT t.nickname team, 
	count(m.winner) * 2 as points,
    count(m.winner) as wins
FROM Matches m
JOIN Teams t
ON m.winner = t.id
WHERE year(m.date) = 2018
GROUP BY m.winner
ORDER BY points DESC;'''

In [117]:
query = '''SELECT CONCAT(p.first_name, ' ' ,p.last_name) name, t.nickname team, p_stats.*
        FROM PlayerMatchStats p_stats
        JOIN Players p
        ON p_stats.player_id = p.id
        JOIN Teams t
        ON p_stats.team_id = t.id;'''

player_df = create_df(query)

In [123]:
print(player_df.groupby(['match_id', 'team_id']))